In [3]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.utils import all_estimators 
from sklearn.base import ClassifierMixin

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [4]:
def load_df(col_cible_type):
    df = pd.read_csv(
        f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False
    )
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df

In [ ]:
import copy

cols_cible_type = ["TurnoutTimeSeconds", "TravelTimeSeconds", "PumpSecondsOnSite"]
cols_Data = [
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "PropertyType", "StopCode"],
]
cols_cible_bins = [
    {
        "bins": np.array([0, 1, 2, 3, 5, 8, 10, 15, 20000]) * 60,
        "labels": [1, 2, 3, 5, 8, 10, 15, 30],
    },
    {
        "bins": np.array([0, 1, 2, 3, 5, 8, 10, 15, 20000]) * 60,
        "labels": [1, 2, 3, 5, 8, 10, 15, 30],
    },
    {
        "bins": np.array([0, 5, 10, 15, 30, 45, 60, 120, 180, 360, 1000000]) * 60,
        "labels": [5, 10, 15, 30, 45, 60, 120, 180, 360, 1000],
    },
]
cols_cible = [
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"],
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"],
    # NumPumpsAttending??
]
# copie profonde, sinon la simple copie fait une copie des références des ss tableaux, et leur modif modifie l'original
cols_cible_minutes = copy.deepcopy(cols_cible)

ignore_classifiers = [
    "CalibratedClassifierCV",
    "CalibratedClassifierCV",
    "CategoricalNB",
    "ClassifierChain",
    "ComplementNB",
    "FixedThresholdClassifier",
    "GaussianProcessClassifier",
    "GradientBoostingClassifier",
    "LabelPropagation",
    "LabelSpreading",
    "LinearSVC",
    "LogisticRegressionCV",
]

CLASSIFIERS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], ClassifierMixin) and (not (est[0] in ignore_classifiers)))
]

for index, name in enumerate(cols_cible_type):
    df = load_df(name)
    # Crée 3 nouvelles target pour chque min/mean/max, par tranche en minutes, pour la cible type actuelle
    for index_cible, col_cible in enumerate(cols_cible[index]):
        bins = cols_cible_bins[index]["bins"]
        print(bins)
        labels = cols_cible_bins[index]["labels"]
        print(labels)
        new_name = col_cible.replace("Seconds", "Minutes")
        print(new_name, col_cible)
        df[new_name] = pd.cut(x=df[col_cible], bins=bins, labels=labels)
        cols_cible_minutes[index][index_cible] = new_name
        print("cols_cible", cols_cible)
        print("cols_cible_minutes", cols_cible_minutes)
    display(df.head(10))

    # df = df[df.CalYear > 6]

    # pd.DataFrame(pd.cut(X['Age'], bins = [0, 12, 18, 30, 50, 65, np.max(df.Age)], labels=['Kid','Adolescent','Adult-','Adult','Adult+','Senior']))
    # df['age_cat'] = pd.cut(x = df['age'], bins = [19,24,28,32,36,40], labels = ['19-24', '25-28', '29-32', '33-36', '37-40'],
    #                    include_lowest = True)

    # X = df[["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
    #               "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"]]

    # plt.boxplot([df.TurnoutTimeSeconds_min, df.TurnoutTimeSeconds_mean, df.TurnoutTimeSeconds_max])
    # plt.show()
    # plt.boxplot([df.TravelTimeSeconds_min, df.TravelTimeSeconds_mean, df.TravelTimeSeconds_max])
    # plt.show()
    # plt.figure(figsize=(5, 10))
    # plt.boxplot([df.PumpSecondsOnSite_min, df.PumpSecondsOnSite_mean, df.PumpSecondsOnSite_max])
    # plt.show()

    # Auto ML
    df_limited = df[df.CalYear > 10]
    print(df_limited.shape)
    # Crée X sans les target
    cols_to_remove = (
        [item for sublist in cols_cible for item in sublist]
        + cols_cible_minutes[index]
        + ["NumPumpsAttending"]
    )
    print("cols_to_remove", cols_to_remove)
    X = df_limited.drop(cols_to_remove, axis=1)
    # Ne conserve que certaines colonnes explicatives
    # Crée une liste des colonnes à conserver
    cols_to_keep = [
        col
        for col in X.columns
        if any(substring in col for substring in cols_Data[index])
    ]
    print("cols_to_keep", cols_to_keep)
    # Conserver uniquement ces colonnes
    X = X[cols_to_keep]

    display(X.head(2))
    print(X.shape)

    for col_cible in cols_cible[index]:
        print(col_cible)
        y = df_limited[col_cible]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        myCLASSIFIERS = CLASSIFIERS
        print(myCLASSIFIERS)
        # myCLASSIFIERS = myCLASSIFIERS[4:6]
        reg = LazyClassifier(
            verbose=2,
            ignore_warnings=False,
            custom_metric=None,
            classifiers=myCLASSIFIERS,
        )
        models, predictions = reg.fit(X_train, X_test, y_train, y_test)
        display(models)
        models.to_csv(
            f"../data/_autoML_classifier_{col_cible}.csv", sep=";", index=True
        )
        # break
    # break

# tester automl avant et après PCA et aussi gridsearch pour comparer
# save du PCA

[      0      60     120     180     300     480     600     900 1200000]
[1, 2, 3, 5, 8, 10, 15, 30]
TurnoutTimeMinutes_min TurnoutTimeSeconds_min
cols_cible [['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max'], ['PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max']]
cols_cible_minutes [['TurnoutTimeMinutes_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max'], ['PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max']]
[      0      60     120     180     300     480     600     900 1200000]
[1, 2, 3, 5, 8, 10, 15, 30]
TurnoutTimeMinutes_mean TurnoutTimeSeconds_mean
cols_cible [['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max'], ['TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max'], ['PumpSecondsOnSite_min',

,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,PropertyType_0,PropertyType_1,PropertyType_2,PropertyType_3,PropertyType_4,PropertyType_5,PropertyType_6,PropertyType_7,PropertyType_8,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,NumPumpsAttending,StopCode_0,StopCode_1,StopCode_2,StopCode_3,StopCode_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max,TurnoutTimeMinutes_min,TurnoutTimeMinutes_mean,TurnoutTimeMinutes_max
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2.00,0,0,0,0,1,0,0,0,1,0,0,1,240.00,390.00,540.00,253.00,253.00,253.00,89.00,89.00,89.00,5,5,5
1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1.00,0,0,0,1,0,0,0,0,1,0,0,1,420.00,420.00,420.00,151.00,151.00,151.00,157.00,157.00,157.00,3,3,3
2,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1.00,0,0,0,1,0,0,0,0,1,0,0,1,720.00,720.00,720.00,108.00,108.00,108.00,102.00,102.00,102.00,2,2,2
3,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,2.00,0,0,0,1,0,0,0,0,1,0,0,1,120.00,120.00,120.00,114.00,128.00,142.00,108.00,113.50,119.00,2,3,3
4,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,2.00,0,0,0,1,1,0,0,0,1,0,0,1,360.00,360.00,360.00,83.00,89.00,95.00,89.00,108.00,127.00,2,2,2
5,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1.00,0,0,0,0,1,0,0,0,1,0,0,1,420.00,420.00,420.00,119.00,119.00,119.00,403.00,403.00,403.00,2,2,2
6,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,2.00,0,0,0,1,0,0,0,0,1,0,0,1,1440.00,1440.00,1440.00,178.00,180.50,183.00,164.00,165.00,166.00,3,5,5
7,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1.00,0,0,0,1,0,0,0,0,1,0,0,1,420.00,420.00,420.00,121.00,121.00,121.00,134.00,134.00,134.00,3,3,3
8,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1.00,0,0,0,1,1,0,0,0,1,0,0,1,780.00,780.00,780.00,110.00,110.00,110.00,187.00,187.00,187.00,2,2,2
9,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1.00,0,0,1,0,0,0,0,0,1,0,0,1,600.00,600.00,600.00,129.00,129.00,129.00,567.00,567.00,567.00,3,3,3


(635615, 45)
cols_to_remove ['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max', 'PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max', 'TurnoutTimeMinutes_min', 'TurnoutTimeMinutes_mean', 'TurnoutTimeMinutes_max', 'NumPumpsAttending']
cols_to_keep ['CalYear', 'HourOfCall_0', 'HourOfCall_1', 'HourOfCall_2', 'HourOfCall_3', 'HourOfCall_4', 'Postcode_district_0', 'Postcode_district_1', 'Postcode_district_2', 'Postcode_district_3', 'Postcode_district_4', 'Month_0', 'Month_1', 'Month_2', 'Month_3', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2']


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2
955912,11,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0
955913,11,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0


(635615, 18)
TurnoutTimeSeconds_min
[('AdaBoostClassifier', <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>), ('BaggingClassifier', <class 'sklearn.ensemble._bagging.BaggingClassifier'>), ('BernoulliNB', <class 'sklearn.naive_bayes.BernoulliNB'>), ('DecisionTreeClassifier', <class 'sklearn.tree._classes.DecisionTreeClassifier'>), ('DummyClassifier', <class 'sklearn.dummy.DummyClassifier'>), ('ExtraTreeClassifier', <class 'sklearn.tree._classes.ExtraTreeClassifier'>), ('ExtraTreesClassifier', <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>), ('GaussianNB', <class 'sklearn.naive_bayes.GaussianNB'>), ('HistGradientBoostingClassifier', <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'>), ('KNeighborsClassifier', <class 'sklearn.neighbors._classification.KNeighborsClassifier'>), ('LinearDiscriminantAnalysis', <class 'sklearn.discriminant_analysis.LinearDiscriminantAnalysis'>), ('LogisticRegression', <class 'sklearn.line

...

ROC AUC couldn't be calculated for DecisionTreeClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.012554769789888533, 'Balanced Accuracy': np.float64(0.002667784501252771), 'ROC AUC': None, 'F1 Score': 0.01151344371410894, 'Time taken': 2.1356091499328613}


 15%|█▌        | 5/33 [01:44<05:01, 10.77s/it]

ROC AUC couldn't be calculated for DummyClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'DummyClassifier', 'Accuracy': 0.015378806352902306, 'Balanced Accuracy': np.float64(0.0020920502092050207), 'ROC AUC': None, 'F1 Score': 0.0004658511352813649, 'Time taken': 0.40317320823669434}


 18%|█▊        | 6/33 [01:45<03:23,  7.53s/it]

ROC AUC couldn't be calculated for ExtraTreeClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'ExtraTreeClassifier', 'Accuracy': 0.012554769789888533, 'Balanced Accuracy': np.float64(0.002667784501252771), 'ROC AUC': None, 'F1 Score': 0.01151344371410894, 'Time taken': 1.2453222274780273}


In [5]:
import autosklearn.regression as autosk_reg
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

cols_cible_type = ["PumpSecondsOnSite", "TurnoutTimeSeconds", "TravelTimeSeconds"]
cols_cible = [
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"],
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"]
    # NumPumpsAttending??
]

def load_df(col_cible_type):
    df = pd.read_csv(f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False)
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df

for index, name in enumerate(cols_cible_type):
    df = load_df(name)
    X = df.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
                  "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    display(X.head(2))
    print(X.shape)
    
    # Auto ML
    df_limited = df[df.CalYear >= 10]
    X = df_limited.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
                  "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    for col_cible in cols_cible[index]:
        print("//////////////////////////////////////////////////////////////////////////////////////////")
        print(col_cible)
        y = df_limited[col_cible]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Créer et entraîner Auto-sklearn pour la régression
        automl = autosk_reg.AutoSklearnRegressor(
            time_left_for_this_task=80000,  # Temps total alloué pour l'entraînement (en secondes)
            per_run_time_limit=6000,  # Temps maximum pour chaque modèle testé
            n_jobs=-1,  # Utilise tous les cœurs disponibles
            ensemble_kwargs = {'ensemble_size': 10},  # Nombre maximal de modèles dans l'ensemble final
            seed=42,
        )

        # Entraînement
        automl.fit(X_train, y_train)
        # Prédiction
        y_pred = automl.predict(X_test)
        # Évaluation des performan ces
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Mean Squared Error: {mse}")
        print(f"R2 Score: {r2}")
        # Afficher un résumé détaillé des modèles et hyperparamètres
        print("\n********** Rapport détaillé des modèles **********\n")
        print(automl.show_models())
        print("\n********** Statistiques d'entraînement **********\n")
        print(automl.sprint_statistics())

ModuleNotFoundError: No module named 'autosklearn'